In [27]:
from utilsGeneral import UtilsGeneral
from utilsModel import UtilsModel
from utilsPlot import UtilsPlot
from KnapSack import KnapSack
from shallowNet.shallowNet import shallowNet, DenseTranspose
import numpy as np
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import datetime


In [14]:
knapSack = KnapSack("100_5_25_1")
utg = UtilsGeneral(knapSack)
utm = UtilsModel(utg)
utp = UtilsPlot(utilsGeneral = utg, utilsModel = utm)
fitness_function = knapSack.Fitness

Best Objective Value: 0


In [15]:
train = utg.generate_training_set(knapSack.Size, 100)
test = utg.generate_training_set(knapSack.Size, 30)

In [10]:

HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.4))
HP_REG_COF = hp.HParam('reg_cof', hp.Discrete([0.002, 0.0002]))
HP_COMPRESSION = hp.HParam('compression', hp.Discrete([0.8, 0.7,0.6]))

METRIC_ACCURACY ="val_root_mean_squared_error"

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_DROPOUT, HP_REG_COF],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )



In [28]:
def train_test_model(hparams):
    model = shallowNet.build(
        input_shape= knapSack.Size,
        compression=0.8,
        reg_cof=hparams[HP_REG_COF],
        dropout=hparams[HP_DROPOUT],
    )
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(train, train, epochs =1, callbacks = [tensorboard_callback])
    loss = model.evaluate(test, test)
    return loss

    

In [29]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [30]:
session_num = 0

for reg_cof in HP_REG_COF.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        hparams = {
            HP_DROPOUT: dropout_rate,
            HP_REG_COF: reg_cof,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'dropout': 0.2, 'reg_cof': 0.0002}
1/1 [==============================] - 0s 2ms/step - loss: 0.7433
--- Starting trial: run-1
{'dropout': 0.4, 'reg_cof': 0.0002}
1/1 [==============================] - 0s 968us/step - loss: 0.7291
--- Starting trial: run-2
{'dropout': 0.2, 'reg_cof': 0.002}
1/1 [==============================] - 0s 997us/step - loss: 0.9726
--- Starting trial: run-3
{'dropout': 0.4, 'reg_cof': 0.002}
1/1 [==============================] - 0s 2ms/step - loss: 0.9899


In [32]:
with tf.Session() as sess:
    print(sess.run(c))

AttributeError: module 'tensorflow' has no attribute 'Session'